In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import mlflow
import mlflow.sklearn
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# 데이터 로딩
data = pd.read_csv('../data/dataset.csv')

In [3]:
# 예측 타겟과 특성 분리
X = data.drop('Default', axis=1)
y = data['Default']

In [4]:
# 범주형과 수치형 컬럼 구분
categorical_cols = ['State', 'BankState', 'NewExist', 'UrbanRural', 'RealEstate']
numerical_cols = ['DisbursementGross', 'GrAppv', 'daysterm']


In [5]:
# 데이터 전처리 파이프라인 구성
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])

In [6]:
# 데이터 전처리 실행
X_processed = preprocessor.fit_transform(X)

In [7]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# 하이퍼파라미터 탐색 공간 설정
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', range(50, 200)),
    'gamma': hp.uniform('gamma', 0, 5)
}

In [8]:
# 최적화를 위한 목적 함수 정의
def objective(params):
    # MLflow에 실험 이름 설정
    mlflow.set_experiment("assignment1")

    # 각 하이퍼파라미터 조합별 실험 시작
    with mlflow.start_run(nested=True):
        # XGBoost 모델 초기화 및 훈련
        model = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False, **params)
        model.fit(X_train, y_train)

        # 예측 확률 및 ROC-AUC 계산
        probs = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, probs)

        # MLflow에 파라미터 및 메트릭 로깅
        mlflow.log_params(params)
        mlflow.log_metric("roc_auc", auc)

        # 목적 함수 결과 반환 (손실 최소화)
        return {'loss': -auc, 'status': STATUS_OK}


In [9]:
# Hyperopt를 사용한 최적화 실행
trials = Trials()
best_params = fmin(
    fn=objective, # 목적 함수
    space=space,    # 탐색 공간
    algo=tpe.suggest,   # 어떻게 탐색할거냐. (최적화 알고리즘)
    max_evals=30,   # 시도 횟수
    trials=trials   # 결과 저장소
)

  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

2025/10/10 17:17:41 INFO mlflow.tracking.fluent: Experiment with name 'assignment1' does not exist. Creating a new experiment.

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:17:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



🏃 View run ambitious-doe-833 at: http://127.0.0.1:5000/#/experiments/1/runs/033af9502d5a406496f7107675748b8e

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1

  3%|▎         | 1/30 [00:02<01:26,  3.00s/trial, best loss: -0.9727997227997228]

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:17:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



🏃 View run thoughtful-fowl-281 at: http://127.0.0.1:5000/#/experiments/1/runs/2b0a3cb0899b4f20a84ed0ac641f7800

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                     

  7%|▋         | 2/30 [00:03<00:40,  1.44s/trial, best loss: -0.9754479754479755]

...

🏃 View run resilient-worm-491 at: http://127.0.0.1:5000/#/experiments/1/runs/074fe10d73ca4b4fbed47fd5921f6830

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

 93%|█████████▎| 28/30 [00:10<00:00,  3.72trial/s, best loss: -0.9797049797049796]

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:17:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:17:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



🏃 View run efficient-cod-912 at: http://127.0.0.1:5000/#/experiments/1/runs/b9900762b3e945c6aeace268e6dd259a

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

 97%|█████████▋| 29/30 [00:11<00:00,  3.47trial/s, best loss: -0.9797049797049796]

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:17:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



🏃 View run clean-toad-645 at: http://127.0.0.1:5000/#/experiments/1/runs/d0d766c2d9fa4ff8b3a67340125cd862

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                      

100%|██████████| 30/30 [00:11<00:00,  2.65trial/s, best loss: -0.9797049797049796]
